In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
lava18_google_play_store_apps_path = kagglehub.dataset_download('lava18/google-play-store-apps')

print('Data source import complete.')


Hi everybody !

In this notebook, I'm gonna analyze Google Play Store datas. While I was analyzing the data, I used Python. This study is my first data analyzing study. If you liked this kernel or it was benefit to you, forgot upvotes!
Good studies. :) <br/>
<br/>
### **CONTENT**

1. [Introduction to data](#1)  <br/>
2. [Cleaning Data](#2)         <br/>
&nbsp;&nbsp;&nbsp;&nbsp; A. [Category](#a)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; B. [Rating](#b)      <br/>
&nbsp;&nbsp;&nbsp;&nbsp; C. [Reviews](#c)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; D. [Size](#d)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; E. [Installs](#e)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; F. [Price](#f)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; G. [Last Updated](#g)  <br/>
3. [Exploratory Data Analysis](#3)  <br/>
&nbsp;&nbsp;&nbsp;&nbsp; A. [Category and Reviews](#a1)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; B. [Category and Installs](#b1)      <br/>
&nbsp;&nbsp;&nbsp;&nbsp; C. [Word Cloud](#c1)   <br/>
&nbsp;&nbsp;&nbsp;&nbsp; D. [Content Rating](#d1)   <br/>

<a id="1"></a>
# 1. INTRODUCTION TO DATA

Firstly let's get to know data.  While I was analyzing the data, I used Pandas library.  <br/>
* info(): It informs about data columns and data types. <br/>
* head(): It returns the first five data. <br/>
* tail(): It returns the last five data. <br/>
* columns : It returns data columns <br/>
* shape : It gives number of rows and columns in a tuble.


In [ ]:
#import library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool
# plotly
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# word cloud library
from wordcloud import WordCloud


In [ ]:
#read to csv
data = pd.read_csv("../input/googleplaystore.csv")

In [ ]:
data.info()

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

We can combine tables to make it easier to see data. For this, we are gonna use "concat function" that is found Pandas library. <br/> <br/>
pd.concat([data frame parameters], axis,ignore_index) : It combines 2 tables. <br/>
**axis** : It adds the tables as horizontal or vertical. If axis equals 0, it adds as horizontal. If axis equals 1, it adds as vertical. <br/>
**ignore_index** : It ignores index values.


In [ ]:
data1 = data.head()
data2 = data.tail()
concat_data = pd.concat([data1,data2],axis=0,ignore_index=True)
concat_data

<a id="2"></a>
# 2. Cleaning Data

Dataset can contain missing data, numerical string value, various cues. If we can clean them, we can make easy our analysis.  <br/>

Let's have some fun. :)

<a id="a"></a>
## Category

In [ ]:
data['Category'].unique()

'1.9' is wrong value. So I have to see in which rows '1.9' corresponds to.

In [ ]:
data[data['Category'] == '1.9']

We can make NaN category value. At that time we can get a clean data. I used shift() method that is found Pandas library.

In [ ]:
data.loc[10472] = data.loc[10472].shift()
data['App'].loc[10472] = data['Category'].loc[10472]
data['Category'].loc[10472] = np.nan
data.loc[10472]

<a id="b"></a>
## Rating

In [ ]:
data['Rating'].unique()

Data type of Rating is object. If we convert from string to numeric, we can make easy.

In [ ]:
data['Rating'] = pd.to_numeric(data['Rating'], errors='coerce')
data['Rating'].dtype

<a id="c"></a>
## Reviews

In [ ]:
data['Reviews'].unique()

In [ ]:
data[data['Reviews'] =='3.0M']

In [ ]:
data['Reviews'] = data.Reviews.replace("0.0",0)
data['Reviews'] = data.Reviews.replace("3.0M",3000000.0)
data['Reviews'] = data['Reviews'].astype(float)
data['Reviews'].dtype

<a id="d"></a>
## Size

In [ ]:
data['Size'].unique()

Data type of Size is object. I had to convert the column because it contains the application sizes. Firstly I changed 'Varies with device' value with Nan. After, I dropped 'M' and 'k'. I changed from '1000+' to 1000. Finally, I converted float value.

In [ ]:
data['Size'] = data.Size.replace("Varies with device",np.nan)
data['Size'] = data.Size.str.replace("M","000") # All size values became the kilobyte type.
data['Size'] = data.Size.str.replace("k","")
data['Size'] = data.Size.replace("1,000+",1000)
data['Size'] =data['Size'].astype(float)
data['Size'].dtype

<a id="e"></a>
## Installs

In [ ]:
data['Installs'].unique()

Data type of Size is object. I'm gonna make similar processes, which I made the in 'Size'.

In [ ]:
data['Installs'] = data.Installs.str.replace(",","")
data['Installs'] = data.Installs.str.replace("+","")
data['Installs'] = data.Installs.replace("Free",np.nan)
data['Installs'] = data['Installs'].astype(float)
data['Installs'].dtype

<a id="f"></a>
## Price

In [ ]:
data['Price'].unique()

Data type of Price is object. I have made similar processes, which I made the in 'Size'.

In [ ]:
data['Price'] = data.Price.replace("Everyone",np.nan)
data['Price'] = data.Price.str.replace("$","").astype(float)
data['Price'].dtype

<a id="g"></a>
## Last Updated

In [ ]:
data['Last Updated'].unique()

Data type of Last Uptated is object. I converted from string to date type.

In [ ]:
data['Last Updated'] = pd.to_datetime(data['Last Updated'])
data['Last Updated']

<a id="3"></a>
# Exploratory Data Analysis

After, I prepared to analyze our data, somewhat let's explore the datas. :)

* corr() : It returns correlation.
* describe (): It returns number of entries, average of entries, outlier values, standart deviation, minimum and maximum entry.

In [ ]:
data.corr()

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(12, 12))
sns.heatmap(data.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)
plt.show()

In [ ]:
data.describe()

<a id="a1"></a>
## Category and Reviews

In [ ]:
category_list = list(data['Category'].unique())
category_review = []
for i in category_list:
    x = data[data['Category'] == i]
    if(len(x)!=0):
        review = sum(x.Reviews)/len(x)
        category_review.append(review)
    else:
        review = sum(x.Reviews)
        category_review.append(review)
#sorting
data_category_reviews = pd.DataFrame({'category': category_list,'review':category_review})
new_index = (data_category_reviews['review'].sort_values(ascending=False)).index.values
sorted_data =data_category_reviews.reindex(new_index)

# visualization
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data['category'], y=sorted_data['review'])
plt.xticks(rotation=80)
plt.xlabel("Category")
plt.ylabel("Reviews")
plt.title("Category and Reviews")
plt.show()

<a id="b1"></a>
## Category and Installs

In [ ]:
category_list = list(data['Category'].unique())
category_install = []
for i in category_list:
    x = data[data['Category'] == i]
    if(len(x)!=0):
        install = sum(x.Installs)/len(x)
        category_install.append(install)
    else:
        install = sum(x.Installs)
        category_install.append(install)

#sorting
data_category_install = pd.DataFrame({'category': category_list,'install':category_install})
new_index = (data_category_install['install'].sort_values(ascending=False)).index.values
sorted_data =data_category_install.reindex(new_index)

# visualization
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data['category'], y=sorted_data['install'])
plt.xticks(rotation=80)
plt.xlabel("Category")
plt.ylabel("Install")
plt.title("Category and Install")
plt.show()

<a id="c1"></a>
## Word Cloud

In [ ]:
plt.subplots(figsize=(8,8))
wordcloud = WordCloud(
                          background_color='white',
                          width=512,
                          height=384
                         ).generate(" ".join(data))
plt.imshow(wordcloud)
plt.axis('off')
plt.savefig('graph.png')

plt.show()

<a id="d1"></a>
## Content Rating

In [ ]:
plt.figure(figsize=(10,7))
sns.countplot(data=data, x='Content Rating')
plt.xticks(rotation=80)
plt.title('Content Rating',color = 'blue',fontsize=15)
plt.show()

<a id="4"></a>
# Conclusion

This is the end of the story. I hope It benefits to you. You can visualize with a lot of different model. Actually, I thought that EDA study could be boring. But It's important for ML models. I'm definitely gonna myself about this subject. You can help with your comments. Wish to see you with different datasets.

**Thank you for your votes and comments **